# 5. 수집된 데이터 탐색
## 5-1. 스파크 세션 생성
### 5-1-1. 스파크 객체를 생성하는 코드를 작성하고, Shift+Enter 로 스파크 버전을 확인합니다

In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# 코어 스파크 라이브러리를 임포트 합니다
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

spark = (
    SparkSession
    .builder
    .appName("Data Engineer Training Course")
    .config("spark.sql.session.timeZone", "Asia/Seoul")
    .getOrCreate()
)

# 노트북에서 테이블 형태로 데이터 프레임 출력을 위한 설정을 합니다
from IPython.display import display, display_pretty, clear_output, JSON
spark.conf.set("spark.sql.repl.eagerEval.enabled", True) # display enabled
spark.conf.set("spark.sql.repl.eagerEval.truncate", 100) # display output columns size
spark

### 5-1-2. 수집된 테이블을 데이터프레임으로 읽고, 스키마 및 데이터 출력하기

In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
user25 = spark.read.parquet("user/20201025")
user25.printSchema()
user25.show(truncate=False)
display(user25)

purchase25 = spark.read.parquet("purchase/20201025")
purchase25.createOrReplaceTempView("purchase25")
purchase25.printSchema()
display(purchase25)

accesslog = spark.read.option("inferSchema", "true").json("access/20201025")
accesslog.createOrReplaceTempView("accesslog")
access25 = spark.sql("select a_id, a_tag, a_time, a_timestamp, a_uid from accesslog")
access25.createOrReplaceTempView("access25")
access25.printSchema()
display(access25)

## 5-2. 수집된 고객, 매출 및 접속 임시 테이블 생성
### 5-2-1. 데이터프레임을 이용하여 임시테이블 생성하기

In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
user25.createOrReplaceTempView("user25")
purchase25.createOrReplaceTempView("purchase25")
access25.createOrReplaceTempView("access25")
spark.sql("show tables '*25'")

## 5-3. SparkSQL을 이용하여 테이블 별 데이터프레임 생성하기
### 5-3-1. 아래에 비어있는 조건을 채워서 올바른 코드를 작성하세요

In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
u_signup_condition = "u_signup >= '20201025' and u_signup < '20201026'"
user = spark.sql("select u_id, u_name, u_gender from user25").where(u_signup_condition)
user.createOrReplaceTempView("user")
display(user)

p_time_condition = "p_time >= '2020-10-25 00:00:00' and p_time < '2020-10-26 00:00:00'"
purchase = spark.sql("select from_unixtime(p_time) as p_time, p_uid, p_id, p_name, p_amount from purchase25").where(p_time_condition)
purchase.createOrReplaceTempView("purchase")
display(purchase)

access = spark.sql("select a_id, a_tag, a_timestamp, a_uid from access25")
access.createOrReplaceTempView("access")
display(access)

spark.sql("show tables")

## 5-4. 생성된 테이블을 SQL 문을 이용하여 탐색하기
### 5-4-1. 한 쪽의 성별('남' 혹은 '여')을 가진 목록을 출력하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe user")
spark.sql("select * from user")
whereCondition = "u_gender = '남'"
spark.sql("select * from user").where(whereCondition)

### 5-4-2. 상품금액이 200만원을 초과하는 매출 목록을 출력하세요

In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe purchase")
spark.sql("select * from purchase")
selectClause = "select * from purchase where p_amount > 2000000"
spark.sql(selectClause)

### 5-4-3. GroupBy 구문을 이용하여 로그인, 로그아웃 횟수를 출력하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
spark.sql("describe access")
spark.sql("select * from access")
groupByClause="select a_id, count(a_id) from access group by a_id"
spark.sql(groupByClause)

# 6. 기본 지표 생성
## 6-1. DAU (Daily Activer User) 지표를 생성하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
display(access)
distinctAccessUser = "select count(distinct a_uid) as DAU from access"
dau = spark.sql(distinctAccessUser)
display(dau)
v_dau = dau.collect()[0]["DAU"]

## 6-2. DPU (Daily Paying User) 지표를 생성하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)

display(purchase)
distinctPayingUser = "select count(distinct p_uid) as PU from purchase"
pu = spark.sql(distinctPayingUser)
display(pu)
v_pu = pu.collect()[0]["PU"]

## 6-3. DR (Daily Revenue) 지표를 생성하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
display(purchase)
sumOfDailyRevenue = "select sum(p_amount) as DR from purchase"
dr = spark.sql(sumOfDailyRevenue)
display(dr)
v_dr = dr.collect()[0]["DR"]

## 6-4. ARPU (Average Revenue Per User) 지표를 생성하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
# print("ARPU : {}".format(<유저당 매출 금액 계산식>))
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")

## 6-5. ARPPU (Average Revenue Per Paying User) 지표를 생성하세요


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")

# 7. 고급 지표 생성
## 7-1. 디멘젼 테이블을 설계 합니다
## 7-2. 오픈 첫 날 접속한 모든 고객 및 접속 횟수를 가진 데이터프레임을 생성합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
access.printSchema()
countOfAccess = "select a_uid, count(a_uid) as a_count from access where a_id = 'login' group by a_uid"
accs = spark.sql(countOfAccess)
display(accs)

## 7-3. 일 별 이용자 별 총 매출 금액과, 구매 횟수를 가지는 데이터프레임을 생성합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
purchase.printSchema()
sumOfCountAndAmount = "select p_uid, sum(p_amount) as p_amount, count(p_uid) as p_count from purchase group by p_uid"
amts = spark.sql(sumOfCountAndAmount)
display(amts)

## 7-4. 이용자 정보와 구매 정보와 조인합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
accs.printSchema()
amts.printSchema()
joinCondition = accs["a_uid"] == amts["p_uid"]
joinHow = "left_outer"
dim1 = accs.join(amts, joinCondition, joinHow)
dim1.printSchema()
display(dim1.orderBy(asc("a_uid")))

## 7-5. 고객 정보를 추가합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim1.printSchema()
user.printSchema()
joinCondition = dim1["a_uid"] == user["u_id"]
joinHow = "left_outer"
dim2 = dim1.join(user, joinCondition, joinHow)
dim2.printSchema()
display(dim2.orderBy(asc("a_uid")))

## 7-6. 중복되는 ID 컬럼은 제거하고, 숫자 필드에 널값은 0으로 기본값을 넣어줍니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim2.printSchema()
dim3 = dim2.drop("p_uid", "u_id")
fillDefaultValue = {"p_amount":0, "p_count":0}
dim4 = dim3.na.fill(fillDefaultValue)
dim4.printSchema()
display(dim4.orderBy(asc("a_uid")))

## 7-7. 생성된 유저 테이블을 재사용 가능하도록 컬럼 명을 변경합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dim4.printSchema()
dim5 = (
    dim4
    .withColumnRenamed("a_uid", "d_uid")
    .withColumnRenamed("a_count", "d_acount")
    .withColumnRenamed("p_amount", "d_pamount")
    .withColumnRenamed("p_count", "d_pcount")
    .withColumnRenamed("u_name", "d_name")
    .withColumnRenamed("u_gender", "d_gender")
   .drop("a_uid", "a_count", "p_amount", "p_count", "u_name", "u_gender")
   .select("d_uid", "d_name", "d_gender", "d_acount", "d_pamount", "d_pcount")
)
display(dim5.orderBy(asc("d_uid")))

## 7-8. 최초 구매 유저 정보를 추가합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
purchase.printSchema()
selectFirstPurchaseTime = "select p_uid, min(p_time) as p_time from purchase group by p_uid"

first_purchase = spark.sql(selectFirstPurchaseTime)
dim6 = dim5.withColumn("d_first_purchase", lit(None))
dim6.printSchema()

exprFirstPurchase = expr("case when d_first_purchase is null then p_time else d_first_purchase end")

dim7 = (
    dim6.join(first_purchase, dim5.d_uid == first_purchase.p_uid, "left_outer")
    .withColumn("first_purchase", exprFirstPurchase)
    .drop("d_first_purchase", "p_uid", "p_time")
    .withColumnRenamed("first_purchase", "d_first_purchase")
)
    
dimension = dim7.orderBy(asc("d_uid"))
dimension.printSchema()
display(dimension)

## 7-9. 생성된 디멘젼을 저장소에 저장합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
dimension.printSchema()
target_dir="dim_users/dt=20201025"
dimension.write.mode("overwrite").parquet(target_dir)

## 7-10. 생성된 디멘젼을 다시 읽어서 출력합니다


In [ ]:
# 여기에 실습 코드를 작성하고 실행하세요 (Shift+Enter)
newDimension = spark.read.parquet(target_dir)
newDimension.printSchema()
display(newDimension)

In [ ]:
# 최종 결과가 아래의 값과 일치하는지 확인합니다
# DAU:5, PU:4, DR: 12200000, ARPU: 2440000.0, ARPPU: 3050000.0
print("+------------------+")
print("|              DAU |")
print("+------------------+")
print("|                {} |".format(v_dau))
print("+------------------+")
print("+------------------+")
print("|               PU |")
print("+------------------+")
print("|                {} |".format(v_pu))
print("+------------------+")
print("+------------------+")
print("|               DR |")
print("+------------------+")
print("|         {} |".format(v_dr))
print("+------------------+")
print("+------------------+")
print("|             ARPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_dau))
print("+------------------+")
print("+------------------+")
print("|            ARPPU |")
print("+------------------+")
print("|        {} |".format(v_dr / v_pu))
print("+------------------+")